In [20]:
import sys
import os
sys.path.append(os.path.dirname(os.path.abspath(os.path.dirname('__file__'))))
import jReversion as jR
from LDOI import BooleanDOI_processing as BDOIp
from LDOI import qm
import pandas as pd
import numpy as np
import networkx as nx

In [21]:
def gen_random_boolean_network(n=30, k=2, alpha=1.0, bias=0.5, self_loops=False):
    '''
    Generate a random network with k-in degree
    The code is originated from the read_ndetwork() by Colin Campbell.
    '''
    def clean_states(x):
        #cleans binary representation of node input states
        out=x[2:]                                                               # Strip leading 0b
        return '0'*(len(inf)-len(out))+out                                      # Append leading 0's as needed

    while True:
        g = nx.generators.directed.random_k_out_graph(n=n, k=k, alpha=alpha, self_loops=self_loops, seed=None).reverse()
        
        input_node = [node for node in g.nodes if g.in_degree(node) == 0]
        output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        if len(input_node) == 0:              # at least one input node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.in_edges(random_node)))
            input_node = [node for node in g.nodes if g.in_degree(node) == 0]
            
        if len(output_node) == 0:            # at least one output node
            random_node = random.randrange(n)
            g.remove_edges_from(list(g.out_edges(random_node)))
            output_node = [node for node in g.nodes if g.out_degree(node) == 0]
        
        if len(g.subgraph(input_node + output_node).edges) > 0: continue # output nodes are connected to input nodes
                    
        if nx.is_weakly_connected(g): break

    for n in g.nodes:
        inf = list(g.predecessors(n))
        if len(inf) > 0: 
            g.nodes[n]['update_nodes'] = inf.copy()
            g.nodes[n]['update_rules'] = {}

            bool_states = map(bin,range(2**len(inf)))
            bool_states = map(clean_states,bool_states)
            canalizing_variable = random.randrange(len(inf))
            canalizing_value = int(random.random() < 0.5)
            canalized_value = int(random.random() < 0.5)
            for j in bool_states:
                if j[canalizing_variable] == str(canalizing_value):
                    g.nodes[n]['update_rules'][j] = canalized_value
                else: 
                    g.nodes[n]['update_rules'][j] = int(random.random() < bias)    # Store outcome for every possible input
        else:
#             g.add_edge(n, n)
            g.nodes[n]['update_nodes'] = [n]
            g.nodes[n]['update_rules'] = {'0': 0, '1': 1}

    return g,list(g.nodes)

In [22]:
def node_classification(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex, constant_nodes):
    TempGIOW = jR.get_input_output_relation(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex,
                                            constant_nodes=[])
    LDOIs = TempGIOW['LDOIs']
    GeneLDOIs = TempGIOW['gene_LDOIs']
    Conflicts = TempGIOW['conflicts']
    GeneConflicts = TempGIOW['gene_conflicts']
    IORelation = TempGIOW['io_relation']
    GRemained = TempGIOW['G_remained']

    R0 = jR.identifying_r0_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
    # DM = jR.identifying_disconnecting_mutations(GExpanded, InputNodes, OutputNodes, Mapping, InverseMapping)
    R1 = jR.identifying_r1_mutations_ldoi(GExpanded, output_nodes_ex, mapping, inverse_mapping, input_conditions, IORelation)
    #     IIDC = jR.identifying_input_independent_canalizing_mutations(GExpanded, OutputNodes, Mapping, InverseMapping)
    #     UN = jR.identifying_input_unreachable_nodes(GExpanded, OutputNodes, Mapping, InverseMapping, InputConditions)
    RN = jR.identifying_rn_mutations(GExpanded, mapping, inverse_mapping, input_nodes_ex, output_nodes_ex, input_conditions,
                                     IORelation,
                                     R0['ineffective_mutations'], R1['r1_mutations'])

    NodeList = set(read_nodes_dict.values())
    NodeList.difference_update(input_nodes)
    NodeList.difference_update(output_nodes)
    C0, C1, C2, C3 = [], [], [], []
    for NODE in NodeList:
        negNODE = '~' + NODE
        if R0['ineffective'][NODE] and R0['ineffective'][negNODE]:
    #             nodeClass = 'C0'
            C0.append(NODE)
        elif NODE in RN['rn_mutations']:
            if RN['rn_mutations'][NODE] == 'R1':
    #                 nodeClass = 'C1'
                C1.append(NODE)
    #                 canalizing = IIDC['iid_canalizing'][NODE]
    #                 unreachable = UN['input_unreachable'][NODE]
            else:
    #                 nodeClass = 'C2'
                C2.append(NODE)
    #                 canalizing = IIDC['iid_canalizing'][NODE]
    #                 unreachable = UN['input_unreachable'][NODE]
        elif negNODE in RN['rn_mutations']:
            if RN['rn_mutations'][negNODE] == 'R1':
    #                 nodeClass = 'C1'
                C1.append(NODE)
    #                 canalizing = IIDC['iid_canalizing'][NODE]
    #                 unreachable = UN['input_unreachable'][NODE]
        else:
    #             nodeClass = 'C3'
            C3.append(NODE)
    #             canalizing = IIDC['iid_canalizing'][NODE]
    #             unreachable = UN['input_unreachable'][NODE]
    
    return C0, C1, C2, C3

In [23]:
g, read_nodes = gen_random_boolean_network(n=10, k=2, alpha=1.0, bias=0.5, self_loops=False)
mapping = {}  # nodename to number index
inverse_mapping = {}  # number index to nodename
read_nodes_dict = {}
inverse_read_nodes_dict = {}
prefix, suffix = 'n', 'n'
for i, node in enumerate(read_nodes):
    index = prefix + str(i) + suffix
    mapping[str(node)] = index
    inverse_mapping[index] = str(node)
    mapping['~' + str(node)] = '~' + index
    inverse_mapping['~' + index] = '~' + str(node)
    read_nodes_dict[i] = str(node)
    inverse_read_nodes_dict[str(node)] = i

input_nodes = [node for node in g.nodes if g.in_degree(node) == 0]
output_nodes = [node for node in g.nodes if g.out_degree(node) == 0]

num_inputs = len(input_nodes)
num_input_conditions = 2 ** num_inputs
input_conditions = np.ndarray((num_inputs, 2), dtype=object)
for idx, input_node in enumerate(input_nodes):
    input_conditions[idx, 0] = '~' + str(input_node)
    input_conditions[idx, 1] = str(input_node)


output_nodes_ex = []
output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()

GExpanded = BDOIp.Get_expanded_network(g, prefix=prefix, suffix=suffix)

print(len(input_nodes))
print(len(output_nodes))

1
2


In [25]:
%timeit node_classification(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex, constant_nodes=[])

15.5 ms ± 57.3 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [26]:
g, read_nodes = gen_random_boolean_network(n=100, k=2, alpha=1.0, bias=0.5, self_loops=False)
mapping = {}  # nodename to number index
inverse_mapping = {}  # number index to nodename
read_nodes_dict = {}
inverse_read_nodes_dict = {}
prefix, suffix = 'n', 'n'
for i, node in enumerate(read_nodes):
    index = prefix + str(i) + suffix
    mapping[str(node)] = index
    inverse_mapping[index] = str(node)
    mapping['~' + str(node)] = '~' + index
    inverse_mapping['~' + index] = '~' + str(node)
    read_nodes_dict[i] = str(node)
    inverse_read_nodes_dict[str(node)] = i

input_nodes = [node for node in g.nodes if g.in_degree(node) == 0]
output_nodes = [node for node in g.nodes if g.out_degree(node) == 0]

num_inputs = len(input_nodes)
num_input_conditions = 2 ** num_inputs
input_conditions = np.ndarray((num_inputs, 2), dtype=object)
for idx, input_node in enumerate(input_nodes):
    input_conditions[idx, 0] = '~' + str(input_node)
    input_conditions[idx, 1] = str(input_node)


output_nodes_ex = []
output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()

GExpanded = BDOIp.Get_expanded_network(g, prefix=prefix, suffix=suffix)

print(len(input_nodes))
print(len(output_nodes))

1
26


In [27]:
%timeit node_classification(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex, constant_nodes=[])

3.05 s ± 29.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [28]:
g, read_nodes = gen_random_boolean_network(n=1000, k=2, alpha=1.0, bias=0.5, self_loops=False)
mapping = {}  # nodename to number index
inverse_mapping = {}  # number index to nodename
read_nodes_dict = {}
inverse_read_nodes_dict = {}
prefix, suffix = 'n', 'n'
for i, node in enumerate(read_nodes):
    index = prefix + str(i) + suffix
    mapping[str(node)] = index
    inverse_mapping[index] = str(node)
    mapping['~' + str(node)] = '~' + index
    inverse_mapping['~' + index] = '~' + str(node)
    read_nodes_dict[i] = str(node)
    inverse_read_nodes_dict[str(node)] = i

input_nodes = [node for node in g.nodes if g.in_degree(node) == 0]
output_nodes = [node for node in g.nodes if g.out_degree(node) == 0]

num_inputs = len(input_nodes)
num_input_conditions = 2 ** num_inputs
input_conditions = np.ndarray((num_inputs, 2), dtype=object)
for idx, input_node in enumerate(input_nodes):
    input_conditions[idx, 0] = '~' + str(input_node)
    input_conditions[idx, 1] = str(input_node)


output_nodes_ex = []
output_nodes_ex.extend([read_nodes_dict[idx] for idx in output_nodes])
output_nodes_ex.extend(['~'+read_nodes_dict[idx] for idx in output_nodes])
input_nodes_ex = input_conditions.reshape(num_inputs * 2, ).tolist()

GExpanded = BDOIp.Get_expanded_network(g, prefix=prefix, suffix=suffix)

print(len(input_nodes))
print(len(output_nodes))

1
341


In [29]:
%timeit node_classification(GExpanded, mapping, inverse_mapping, input_conditions, output_nodes_ex, constant_nodes=[])

2h 9min 30s ± 25.9 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
